In [2]:
#필요한 패키지 임포트
import pandas as pd
import datetime as datetime
import os
import glob 
from bs4 import BeautifulSoup
import json
from urllib.request import urlopen

In [56]:
def mergeStock(stockNumber):
    """
    기존에 파일이 없는 신규 상장의 경우를 고려해서 try와 except를 사용 
    """
    try:
        tempPath ="/Users/choosunsick/Desktop/Korea_Stocks/Korea_Stocks_since_2018/"  + stockNumber
        stockData = pd.read_csv(tempPath, index_col=0, parse_dates=True, dayfirst=True)
        tempPath = "/Users/choosunsick/Desktop/Korea_Stocks/temp/" + stockNumber 
        stockData_new = pd.read_csv(tempPath, index_col=0, parse_dates=True, dayfirst=True)
        stockData_new = stockData_new.sort_index()
        stockData_new.columns = ['Open','High','Low','Close','Volume','Adj Close']
        stockData_new = stockData_new[['Open','High','Low','Close','Volume','Adj Close']]
        stockData = stockData.append(stockData_new,sort=True)
        stockData = stockData[~stockData.index.duplicated(keep='last')]
        stockData = stockData.sort_index()
        stockData = stockData.fillna(0.0).astype(int)
        stockData = stockData[['Open','High','Low','Close','Volume','Adj Close']]
        savename = "/Users/choosunsick/Desktop/Korea_Stocks/Korea_Stocks_since_2018/"+stockNumber
        stockData.to_csv(savename,index=True)
    except:
        tempPath = "/Users/choosunsick/Desktop/Korea_Stocks/temp/" + stockNumber 
        stockData_new = pd.read_csv(tempPath, index_col=0, parse_dates=True, dayfirst=True)
        stockData_new = stockData_new.sort_index()
        stockData = stockData.fillna(0.0).astype(int)
        stockData_new.columns = ['Open','High','Low','Close','Volume','Adj Close']
        stockData_new = stockData_new[['Open','High','Low','Close','Volume','Adj Close']]
        savename = "/Users/choosunsick/Desktop/Korea_Stocks/Korea_Stocks_since_2018/"+stockNumber
        stockData_new.to_csv(savename,index=True)


In [50]:
def merge_about_2000(stockNumber):
    try:
        tempPath ="/Users/choosunsick/Desktop/Korea_Stocks/Korea_Stocks_Full/" + stockNumber
        stockData = pd.read_csv(tempPath, index_col=0, parse_dates=True, dayfirst=True)
        stockData = stockData[['Open','High','Low','Close','Volume','Adj Close']]
        stockData = stockData.sort_index()
        tempPath = "/Users/choosunsick/Desktop/Korea_Stocks/temp/" + stockNumber 
        stockData_new = pd.read_csv(tempPath, index_col=0, parse_dates=True, dayfirst=True)
        stockData_new = stockData_new.sort_index()
        stockData_new.columns = ['Open','High','Low','Close','Volume','Adj Close']
        stockData_new = stockData_new[['Open','High','Low','Close','Volume','Adj Close']]
        stockData = stockData.append(stockData_new,sort=True)
        stockData = stockData[~stockData.index.duplicated(keep='last')]
        stockData = stockData.fillna(0.0).astype(int)
        stockData = stockData.sort_index()
        stockData = stockData[['Open','High','Low','Close','Volume','Adj Close']]
        savename = "/Users/choosunsick/Desktop/Korea_Stocks/Korea_Stocks_Full/"+stockNumber
        stockData.to_csv(savename,index=True)
    except:
        tempPath ="/Users/choosunsick/Desktop/Korea_Stocks/temp/"  + stockNumber
        stockData = pd.read_csv(tempPath, index_col=0, parse_dates=True, dayfirst=True)
        stockData = stockData.sort_index()
        stockData = stockData.fillna(0.0).astype(int)
        stockData = stockData[['Open','High','Low','Close','Volume','Adj Close']]
        savename="/Users/choosunsick/Desktop/Korea_Stocks/Korea_Stocks_Full/"+stockNumber
        stockData.to_csv(savename,index=True)

In [61]:
#함수의 개별주식별 사용법 
#mergeStock("054340.csv")
merge_about_2000(".csv")

In [40]:
"""
함수를 사용하는 예시 코드
다운받아진 파일들의 목록을 리스트화해서 for 문으로 함수에 인자로 전달합니다. 
"""
list1=os.listdir("/Users/choosunsick/Desktop/Korea_Stocks/Korea_Stocks_since_2018/")
list1=[s for s in list1 if "csv" in s]
for i in list1:
    merge_about_2000(i)

FileNotFoundError: File b'/Users/choosunsick/Desktop/Korea_Stocks/temp/005980.csv' does not exist

# 동기식으로 다운 받을 경우

In [43]:
def extracting_stock_from_yahoo(stock_number_text):
    """
    이 함수는 야후에서 코스피 주식자료를 추출하는 함수입니다.
    """
    url = 'https://finance.yahoo.com/quote/'+str(stock_number_text)+'.KS/history?p='+ str(stock_number_text) + '.KS'
    print(url)
    html = urlopen(url)
    soup4 = BeautifulSoup(html, "html.parser")
    temp = soup4.text.strip()
    temp1 = temp.split('\"HistoricalPriceStore\":')[1]
    temp2 = temp1.split('],"isPending":false,"')[0]
    temp3 = json.dumps(temp2)
    temp4 = temp3.replace('\\','')[11:]
    yahoo_json = json.loads((temp4[:len(temp4)-1]+']'))
    return yahoo_json

def KS_extracting_stock_data(stock_number_text):
    temp = extracting_stock_from_yahoo(stock_number_text)
    temp = json.dumps(temp)
    test = pd.read_json(temp, orient='records')
    # .fillna(0.0).astype(int) 이건 정수형으로 만들기 위한 것입니다.
    test2 = pd.concat([test['open'][:30], test['high'][:30],test['low'][:30],test['close'][:30],test['volume'][:30],test['adjclose'][:30]], axis=1).fillna(0.0).astype(int)
    test2=test2.set_index(test['date'][:30])
    test2.index=test2.index[:].strftime("%Y-%m-%d")
    savename="/Users/choosunsick/Desktop/Korea_Stocks/temp/" +stock_number_text+".csv"
    test2.to_csv(savename,index=True)

In [44]:
def KQ_extracting_stock_from_yahoo(stock_number_text):
    """
    이 함수는 야후에서 코스닥 주식자료를 추출하는 함수입니다.
    """
    url = 'https://finance.yahoo.com/quote/'+str(stock_number_text)+'.KQ/history?p='+ str(stock_number_text) + '.KQ'
    print(url)
    html = urlopen(url)
    soup4 = BeautifulSoup(html, "html.parser")
    temp = soup4.text.strip()
    temp1 = temp.split('\"HistoricalPriceStore\":')[1]
    temp2 = temp1.split('],"isPending":false,"')[0]
    temp3 = json.dumps(temp2)
    temp4 = temp3.replace('\\','')[11:]
    yahoo_json = json.loads((temp4[:len(temp4)-1]+']'))
    return yahoo_json

def KQ_extracting_stock_data(stock_number_text):
    temp = KQ_extracting_stock_from_yahoo(stock_number_text)
    temp = json.dumps(temp)
    temp = pd.read_json(temp, orient='records')
    temp2 = pd.concat([temp['open'][:30], temp['high'][:30],temp['low'][:30],temp['close'][:30],temp['volume'][:30],temp['adjclose'][:30]], axis=1).fillna(0.0).astype(int)
    temp2=temp2.set_index(temp['date'][:30])
    temp2.index=temp2.index[:].strftime("%Y-%m-%d")
    savename="/Users/choosunsick/Desktop/Korea_Stocks/temp/" +stock_number_text+".csv"
    temp2.to_csv(savename,index=True)

In [53]:
KS_extracting_stock_data("015230")

https://finance.yahoo.com/quote/015230.KS/history?p=015230.KS


In [54]:
KQ_extracting_stock_data("054340")

https://finance.yahoo.com/quote/054340.KQ/history?p=054340.KQ
